# Convert existing PyTorch code to Ray AIR

Official pytorch tutorial: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

## Unmodified PyTorch code

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.mps.is_available()
    else "cpu"
)
print("Using {} device".format(device))

Using cuda device


In [6]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1} \n -----------------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer)
    test_epoch(test_dataloader, model, loss_fn)

print("Done")

Epoch 1 
 -----------------------------------------
loss: 2.320704 [    0/60000]
loss: 2.300536 [ 6400/60000]
loss: 2.280033 [12800/60000]
loss: 2.263206 [19200/60000]
loss: 2.254644 [25600/60000]
loss: 2.229671 [32000/60000]
loss: 2.230517 [38400/60000]
loss: 2.202650 [44800/60000]
loss: 2.197895 [51200/60000]
loss: 2.160965 [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 2.155929 

Epoch 2 
 -----------------------------------------
loss: 2.171209 [    0/60000]
loss: 2.158050 [ 6400/60000]
loss: 2.098881 [12800/60000]
loss: 2.110374 [19200/60000]
loss: 2.064276 [25600/60000]
loss: 2.007083 [32000/60000]
loss: 2.031127 [38400/60000]
loss: 1.954005 [44800/60000]
loss: 1.958551 [51200/60000]
loss: 1.881614 [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.878456 

Epoch 3 
 -----------------------------------------
loss: 1.914828 [    0/60000]
loss: 1.881270 [ 6400/60000]
loss: 1.761325 [12800/60000]
loss: 1.799968 [19200/60000]
loss: 1.698061 [25600/60000]
loss: 1.643423 

In [10]:
torch.save(model.state_dict(), "models/01_pytorch_MNIST.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Wrapper function for production code

In [11]:
def train_fn():
    batch_size = 64
    lr = 1e-3
    epochs = 5

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    # get cpu or gpu device for training
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using {} device".format(device))

    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        print(f"Epoch {t+1} \n -----------------------------------------")
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_epoch(test_dataloader, model, loss_fn)
    
    print("Done")
    
    

In [12]:
train_fn()

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1 
 -----------------------------------------
loss: 2.308989 [    0/60000]
loss: 2.294538 [ 6400/60000]
loss: 2.274389 [12800/60000]
loss: 2.263218 [19200/60000]
loss: 2.257612 [25600/60000]
loss: 2.225626 [32000/60000]
loss: 2.233253 [38400/60000]
loss: 2.201899 [44800/60000]
loss: 2.196631 [51200/60000]
loss: 2.165219 [57600/60000]
Test Error: 
 Accuracy: 40.3%, Avg loss: 2.161686 

Epoch 2 
 -----------------------------------------
loss: 2.169200 [    0/60000]
loss: 2.161680 [ 6400/60000]
loss: 2.104584 [12800/60000]
loss: 2.122696 [19200/60000]
loss: 2.079923 [25600/60000]
loss: 2.013402 [32000/60000]
loss: 2.047680 [38400/60000]
loss: 1.963887

## Starting with Ray: Distribute the training

In [13]:
import ray.train as train
from ray.air import session

/home/mpp/.conda/envs/ray/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-05 16:15:38,160	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-05 16:15:38,482	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [14]:
def train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    size = len(dataloader.dataset) // session.get_world_size()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Forward pass
        pred = model(X) 
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)


def test_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    size = len(dataloader.dataset) // session.get_world_size()
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    return test_loss


# Use config dict to configure some hyperparameters
def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]

    # Dynamically adjust the worker batch size according to the number of workers: 
    batch_size_per_worker = batch_size // session.get_world_size()

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)

    # Prepare the data loader for distributed data sharding:
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)

    # Prepare the model for distributed gradient updates:
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        session.report(dict(loss=test_loss))

    
    print("Done")



Now we’ll use Ray Train’s TorchTrainer to kick off the training. Note that we can set the hyperparameters here! In the scaling_config we can also configure how many parallel workers to use and if we want to enable GPU training or not.

In [15]:
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig

trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={
        "batch_size": 64,
        "lr": 1e-3,
        "epochs": 5
    },
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False)
)

result = trainer.fit()
print('Last result: ', result.metrics)

2023-09-05 16:15:48,426	INFO data_parallel_trainer.py:404 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TrainTrainable pid=77763) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TorchTrainer pid=77763) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TorchTrainer pid=77763) Starting distributed worker processes: ['77818 (192.168.33.188)', '77819 (192.168.33.188)']
(RayTrainWorker pid=77818) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=77818) Moving model to device: cpu
(RayTrainWorker pid=77818) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=77819) Done


2023-09-05 16:16:14,337	INFO tune.py:1148 -- Total run time: 25.93 seconds (25.91 seconds for the tuning loop).


Last result:  {'loss': 1.0579689832250023, 'timestamp': 1693905372, 'time_this_iter_s': 4.0605480670928955, 'done': True, 'training_iteration': 5, 'trial_id': 'cd422_00000', 'date': '2023-09-05_16-16-12', 'time_total_s': 22.210975885391235, 'pid': 77763, 'hostname': 'fedora', 'node_ip': '192.168.33.188', 'config': {'train_loop_config': {'batch_size': 64, 'lr': 0.001, 'epochs': 5}}, 'time_since_restore': 22.210975885391235, 'iterations_since_restore': 5, 'experiment_tag': '0'}


## Enable checkpointing to retrieve the model

In [16]:
from ray.air import Checkpoint

def load_data():
    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return training_data, test_data


def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
    
    batch_size_per_worker = batch_size // session.get_world_size()
    
    training_data, test_data = load_data()  # <- this is new!
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)
    
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
    
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        checkpoint = Checkpoint.from_dict(
            dict(epoch=t, model=model.state_dict())
        )
        session.report(dict(loss=test_loss), checkpoint=checkpoint)

    print("Done!")

In [18]:
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
    scaling_config=ScalingConfig(num_workers=20, use_gpu=True),
)
result = trainer.fit()

2023-09-05 16:20:02,159	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. Training has not started in the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 1.0 CPUs and 20.0 GPUs, but the cluster only has 20.0 CPUs and 1.0 GPUs available. Stop the training and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.
2023-09-05 16:21:02,229	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. Training has not started in the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 1.0 CPUs and 20.0 GPUs, but the cluster only has 20.0 CPUs and 1.0 GPUs available. Stop the training and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or 